In [5]:
import requests
import json
import glob, os
from bs4 import BeautifulSoup

In [4]:
d = ""
with open("allDataTBCP1-P4IDS.json") as file:
    d = json.loads(file.read())

In [28]:
data_to_scrape = {}
for fileName in os.listdir("./AllItems"):
    if fileName.endswith(".json"):
        print(os.path.splitext(fileName)[0], fileName)
        with open(os.path.join("./AllItems", fileName)) as file:
            data_to_scrape[os.path.splitext(fileName)[0]] = json.loads(file.read())

onehand onehand.json
ranged ranged.json
relic relic.json
shield shield.json
shirt shirt.json
shoulder shoulder.json
tabard tabard.json
thrown thrown.json
trinket trinket.json
twohand twohand.json
waist waist.json
wrists wrists.json


In [15]:
jsonstr = '{"ID": 34343, "Name": "test", "Obtain": {"NpcID": 25315, "NpcName": "Kil\'jaeden", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "40.0", "Zone": "Sunwell Plateau"}}'
print(json.decode(jsonstr))

AttributeError: module 'json' has no attribute 'decode'

In [26]:
print(data_to_scrape)

3725, 'dura': 45, 'reqlevel': 66, 'sellprice': 26736, 'slotbak': 9}, 'attainable': 0, 'flags2': 24580}, '24869': {'name_enus': 'Blood Knight Bracers', 'quality': 2, 'icon': 'inv_bracer_10', 'screenshot': {}, 'jsonequip': {'appearances': {'0': [25797, '']}, 'armor': 231, 'displayid': 25797, 'dura': 35, 'reqlevel': 64, 'sellprice': 19180, 'slotbak': 9}, 'attainable': 0, 'flags2': 24580}, '31175': {'name_enus': "Blade Dancer's Wristguards", 'quality': 3, 'icon': 'inv_bracer_07', 'screenshot': {}, 'jsonequip': {'appearances': {'0': [43166, '']}, 'armor': 107, 'displayid': 43166, 'dura': 35, 'mleatkpwr': 50, 'reqlevel': 63, 'rgdatkpwr': 50, 'sellprice': 17496, 'slotbak': 9, 'sta': 18}, 'attainable': 0, 'flags2': 24580}, '24612': {'name_enus': 'Laughing Skull Bracelets', 'quality': 2, 'icon': 'inv_bracer_07', 'screenshot': {}, 'jsonequip': {'appearances': {'0': [14081, 'inv_bracer_03']}, 'armor': 49, 'displayid': 14081, 'dura': 25, 'reqlevel': 60, 'sellprice': 11828, 'slotbak': 9}, 'attainab

In [6]:
class Item:
    def __init__(self, id, name, dropID, sourceName, sourceType, dropChance, zone):
        self.ID = id
        self.Name = name
        self.Source = {}
        self.Source["ID"] = dropID
        self.Source["SourceName"] = sourceName
        self.Source["SourceType"] = sourceType
        self.Source["DropChance"] = dropChance
        self.Source["Zone"] = zone

    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)
    def toLuaTable(self):
        string = "{id=" + str(self.ID) + ","
        string += "name=\"" + self.Name.replace('"', '\\"') + "\","
        string += "source={"
        string += "ID=" + str(self.Source["ID"]) + ","
        string += "SourceName=\"" + str(self.Source["SourceName"]).replace('"', '\\"') + "\","
        string += "SourceType=\"" + self.Source["SourceType"] + "\","
        string += "DropChance=\"" + str(self.Source["DropChance"]) + "\","
        string += "Zone=\"" + self.Source["Zone"].replace('"', '\\"') + "\""
        string += "}}"
        return string



In [17]:
print(GetItem(35357, "test").toLuaTable())

{id=35357,name="test",source={ID=18897,SourceName="\"King\" Dond",SourceType="Purchase",DropChance="100",Zone="Nagrand"}}


In [7]:
def get_region(content, start, end = None):
    idx_s = content.index(start)
    idx_e = len(content)
    if end:
        idx_e = content.index(end, idx_s)
    return content[idx_s + len(start): idx_e]

In [8]:

def GetZone(zoneID):
        zUrl = ZoneUrl + str(zoneID)
        pageZone = requests.get(zUrl)
        soupZone = BeautifulSoup(pageZone.content, 'html.parser')
        zoneNameH1 = soupZone.find('h1', class_='heading-size-1')
        zoneName = zoneNameH1.text
        return zoneName

In [9]:
def GetCategory(categoryID, category2ID):
        cUrl = QuestCaregoryUrl + str(category2ID) + "." + str(categoryID)
        pageCat = requests.get(cUrl)
        soupCat = BeautifulSoup(pageCat.content, 'html.parser')
        CatNameH1 = soupCat.find('h1', class_='heading-size-1')
        zoneName = CatNameH1.text.replace(" Quests", "")
        return zoneName

In [15]:
def GetName(scripts, scriptStart, dataStart, dataEnd):
    data_container = None
    for child in scripts:
        content = ''.join(child.contents).replace('\n   ', '').strip()
        if content.startswith(scriptStart):
            data_container = content
            break
    r0 = get_region(data_container, dataStart)
    r1 = get_region(r0, ' ', dataEnd)
    return r1[9:]

In [10]:
def GetQuestName(questID):
    qUrl = QuestUrl + str(questID)
    pageCat = requests.get(qUrl)
    soupCat = BeautifulSoup(pageCat.content, 'html.parser')
    CatNameH1 = soupCat.find('h1', class_='heading-size-1')
    questName = CatNameH1.text
    return questName

In [11]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [27]:
URL = 'https://tbc.wowhead.com/item='
QuestCaregoryUrl = 'https://tbc.wowhead.com/quests='
ZoneUrl = 'https://tbc.wowhead.com/zone='
QuestUrl = 'https://tbc.wowhead.com/quest='

rUrl = 'https://www.wowhead.com/item='

SCRIPT_START = "var tabsRelated = new Tabs({parent: WH.ge('jkbfksdbl4'), trackable: 'Item'});"
DATA_START = 'data:'


def GetItem(itemID, itemName, retail):
    o = Item(itemID, itemName, 0, "", "Unknown", 0, "")
    if (itemID == 0):
        return o
    data_container = None
    url = URL + str(itemID)
    if (retail):
        url = rUrl + str(itemID)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    main = soup.find(id='main-contents')
    scripts = main.findAll('script')
    for child in scripts:
        content = ''.join(child.contents).replace('\n   ', '').strip()
        if content.startswith(SCRIPT_START):
            data_container = content
            if (len(content) < 80):
                return o
            break
    if (data_container == None):
        print("Could not find script data")
        return o
    r0 = get_region(data_container, DATA_START)
    r1 = get_region(r0, ' ', ',\n});')
    acqName = GetName(scripts, SCRIPT_START, "name:", ",").lower()
    
    data = json.loads(r1)
    usefulData = data[0]
    zoneName = "Unknown"
    print(acqName)
    if (acqName == "droppedby"): #Dropped by kill
        dropChance = 0
        for i in data:
            if ('percentOverride' in i):
                dropChance = truncate(i['percentOverride'], 1)
            if ('count' in i):
                if (i['count'] >= 0 and i['outof'] > 0):
                    if (truncate(int(i['count'])/int(i['outof']) * 100, 1) > dropChance):
                        usefulData = i
                        dropChance = truncate(int(usefulData['count'])/int(usefulData['outof']) * 100, 1)
        # Get location
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        o = Item(itemID, itemName, usefulData['id'], usefulData['name'], "Kill", str(dropChance), zoneName)

    elif (acqName == "soldby"): #Sold by npc
        dropChance = 100
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        o = Item(itemID, itemName, usefulData['id'], usefulData['name'], "Purchase", str(dropChance), zoneName)

    elif (acqName == "containedin"): #Contained in cache
        dropChance = 0
        for i in data:
            if ('count' in i):
                if (i['count'] >= 0 and i['outof'] > 0):
                    if (truncate(int(i['count'])/int(i['outof']) * 100, 1) > dropChance):
                        usefulData = i
                        dropChance = truncate(int(usefulData['count'])/int(usefulData['outof']) * 100, 1)
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        o = Item(itemID, itemName, usefulData['id'], usefulData['name'], "Container", str(dropChance), zoneName)
    elif (acqName == "rewardfrom"): #Reward from quest
        if ('category' in usefulData and 'category2' in usefulData):
            zoneName = GetCategory(usefulData['category'], usefulData['category2'])
        o = Item(itemID, itemName, usefulData['id'], GetQuestName(usefulData['id']), "Quest", 0, zoneName)

    elif (acqName == "createdby"): #Crafted
        o = Item(itemID, itemName, usefulData['id'], "", "Recipe", 0, "")

    else:    #No info, backup at retail wowhead.
        o = GetItem(itemID, itemName, True)
    return o


In [29]:
print(GetItem(12590, "test", False).toLuaTable())

Could not find script data
{id=12590,name="test",source={ID=0,SourceName="",SourceType="Unknown",DropChance="0",Zone=""}}


In [18]:
print(allItems.items())

04', 'screenshot': {}, 'jsonequip': {'appearances': {'0': [29191, 'inv_spear_08']}, 'buyprice': 2505080, 'displayid': 29191, 'dmgmax1': 406, 'dmgmin1': 270, 'dmgrange': 0.4, 'dmgtype1': 0, 'dps': 105.63, 'dura': 120, 'mledmgmax': 406, 'mledmgmin': 270, 'mledps': 105.63, 'mlehitrtng': 36, 'mlespeed': 3.2, 'reqfaction': 1011, 'reqlevel': 70, 'reqrep': 7, 'rgdhitrtng': 36, 'sellprice': 501016, 'sheathtype': 2, 'slotbak': 17, 'speed': 3.2, 'sta': 54, 'str': 37}, 'attainable': 0, 'flags2': 24580}), ('29605', {'name_enus': "Champion's Lamellar Shoulders", 'quality': 3, 'icon': 'inv_shoulder_28', 'screenshot': {}, 'jsonequip': {'appearances': {'0': [38177, 'inv_shoulder_22']}, 'armor': 552, 'classes': 2, 'displayid': 38177, 'dmgrange': 0.2, 'dura': 80, 'int': 8, 'itemset': 697, 'maxcount': 1, 'reqlevel': 60, 'slotbak': 3, 'spldmg': 20, 'splheal': 20, 'sta': 14, 'str': 14}, 'attainable': 0, 'flags2': 24580}), ('5963', {'name_enus': 'Barbaric Leggings', 'quality': 2, 'icon': 'inv_pants_12', 'sc

In [103]:
nd = {}
for k, v in d.items():
    print("Scraping class", k) # Class
    nd[k] = {}
    for k2, v2 in v.items():
        for k3, v3 in v2.items():
            ns = k2 + " - " + k3
            print("Scraping Set", ns) # Phase
            nd[k][ns] = {}
            nd[k][ns]["Head"] = GetItem(d[k][k2][k3]["Head"]["itemID"])
            nd[k][ns]["Neck"] = GetItem(d[k][k2][k3]["Neck"]["itemID"])
            nd[k][ns]["Shoulder"] = GetItem(d[k][k2][k3]["Shoulder"]["itemID"])
            nd[k][ns]["Back"] = GetItem(d[k][k2][k3]["Cloak"]["itemID"])
            nd[k][ns]["Chest"] = GetItem(d[k][k2][k3]["Chest"]["itemID"])
            nd[k][ns]["Shirt"] = GetItem(0)
            nd[k][ns]["Tabard"] = GetItem(0)
            nd[k][ns]["Wrists"] = GetItem(d[k][k2][k3]["Wrist"]["itemID"])
            nd[k][ns]["Hands"] = GetItem(d[k][k2][k3]["Gloves"]["itemID"])
            nd[k][ns]["Waist"] = GetItem(d[k][k2][k3]["Waist"]["itemID"])
            nd[k][ns]["Legs"] = GetItem(d[k][k2][k3]["Legs"]["itemID"])
            nd[k][ns]["Feet"] = GetItem(d[k][k2][k3]["Boots"]["itemID"])
            nd[k][ns]["Finger"] = GetItem(d[k][k2][k3]["Ring1"]["itemID"])
            nd[k][ns]["RFinger"] = GetItem(d[k][k2][k3]["Ring2"]["itemID"])
            nd[k][ns]["Trinket"] = GetItem(d[k][k2][k3]["Trinket1"]["itemID"])
            nd[k][ns]["RTrinket"] = GetItem(d[k][k2][k3]["Trinket2"]["itemID"])
            nd[k][ns]["MainHand"] = GetItem(d[k][k2][k3]["MainHand"]["itemID"])
            nd[k][ns]["SecondaryHand"] = GetItem(d[k][k2][k3]["OffHand"]["itemID"])
            nd[k][ns]["Relic"] = GetItem(d[k][k2][k3]["Ranged"]["itemID"])
print("---Done with scrape---")

Scraping class Druid
Scraping Set Tank - Pre-Raid


TypeError: GetItem() missing 1 required positional argument: 'itemName'

In [5]:
nd = {}
for k, v in d.items():
    print("Scraping class", k) # Class
    nd[k] = {}
    for k2, v2 in v.items():
        for k3, v3 in v2.items():
            ns = k2 + " - " + k3
            print("Scraping Set", ns) # Phase
            nd[k][ns] = {}
            nd[k][ns]["Head"] = d[k][k2][k3]["Head"]["itemID"]
            nd[k][ns]["Neck"] = d[k][k2][k3]["Neck"]["itemID"]
            nd[k][ns]["Shoulder"] = d[k][k2][k3]["Shoulder"]["itemID"]
            nd[k][ns]["Back"] = d[k][k2][k3]["Cloak"]["itemID"]
            nd[k][ns]["Chest"] = d[k][k2][k3]["Chest"]["itemID"]
            nd[k][ns]["Shirt"] = 0
            nd[k][ns]["Tabard"] = 0
            nd[k][ns]["Wrists"] = d[k][k2][k3]["Wrist"]["itemID"]
            nd[k][ns]["Hands"] = d[k][k2][k3]["Gloves"]["itemID"]
            nd[k][ns]["Waist"] = d[k][k2][k3]["Waist"]["itemID"]
            nd[k][ns]["Legs"] = d[k][k2][k3]["Legs"]["itemID"]
            nd[k][ns]["Feet"] = d[k][k2][k3]["Boots"]["itemID"]
            nd[k][ns]["Finger"] = d[k][k2][k3]["Ring1"]["itemID"]
            nd[k][ns]["RFinger"] = d[k][k2][k3]["Ring2"]["itemID"]
            nd[k][ns]["Trinket"] = d[k][k2][k3]["Trinket1"]["itemID"]
            nd[k][ns]["RTrinket"] = d[k][k2][k3]["Trinket2"]["itemID"]
            nd[k][ns]["MainHand"] = d[k][k2][k3]["MainHand"]["itemID"]
            nd[k][ns]["SecondaryHand"] = d[k][k2][k3]["OffHand"]["itemID"]
            nd[k][ns]["Relic"] = d[k][k2][k3]["Ranged"]["itemID"]
print("---Done with scrape---")

Scraping class Druid
Scraping Set Tank - Pre-Raid
Scraping Set Tank - P1
Scraping Set Tank - P2
Scraping Set Tank - P3
Scraping Set Tank - P4
Scraping Set Feral - Pre-Raid
Scraping Set Feral - P1
Scraping Set Feral - P2
Scraping Set Feral - P3
Scraping Set Feral - P4
Scraping Set Restoration - Pre-Raid
Scraping Set Restoration - P1
Scraping Set Restoration - P2
Scraping Set Restoration - P3
Scraping Set Restoration - P4
Scraping class Hunter
Scraping Set Beast Mastery - Pre-Raid
Scraping Set Beast Mastery - P1
Scraping Set Beast Mastery - P2
Scraping Set Beast Mastery - P3
Scraping Set Beast Mastery - P4
Scraping Set Survival - Pre-Raid
Scraping Set Survival - P1
Scraping Set Survival - P2
Scraping Set Survival - P3
Scraping Set Survival - P4
Scraping class Mage
Scraping Set Arcane - Pre-Raid
Scraping Set Arcane - P1
Scraping Set Arcane - P2
Scraping Set Arcane - P3
Scraping Set Arcane - P4
Scraping Set Fire - Pre-Raid
Scraping Set Fire - P1
Scraping Set Fire - P2
Scraping Set Fire - P

In [29]:

header = "-------------------------------------**     LibEquippable     **-------------------------------------\n-- This file is autogenerated, please do not edit it manually or fuckups might occur.\n-------------------------------------**  All Rights Reserved  **-------------------------------------\n\nlocal LE = LibStub and LibStub(\"LibEquippable-1.0\", true)\nif not LE  then return end\n\nlocal items = {}\nlocal names = {}\n"

footer = "LE:RegisterDBItems(items)\nLE:RegisterNameDBItems(names)"

path_to_lib_db = "./libs/LibEquippable-1.0/db/"

items_scraped = 1
for i, j in data_to_scrape.items():
    itemsLua = ""
    namesLua = ""
    print("*----------*\nScraping", i, "\n*----------*")
    for k, v in j.items():
        print("Scraping item number", items_scraped, "- ID:", k, "- Name:", v["name_enus"])
        item = GetItem(k, v["name_enus"]).toLuaTable()
        itemLuaString = "items[" + k + "] = " + item + "\n"
        nameLuaString = "names[\"" + v["name_enus"].replace('"', '\\"') + "\"] = " + k + "\n"
        itemsLua += itemLuaString
        namesLua += nameLuaString
        items_scraped += 1
    newFile = open(path_to_lib_db + i + ".lua", "w")
    newFile.write(header + "\n\n-- Item Data\n" + itemsLua + "\n\n-- Name Data\n" + namesLua + "\n\n" + footer)
    newFile.close()

 Gossamer Bracers
Scraping item number 6180 - ID: 10094 - Name: Gothic Plate Vambraces
Scraping item number 6181 - ID: 14834 - Name: Tyrant's Armguards
Scraping item number 6182 - ID: 15610 - Name: Bonelink Bracers
Scraping item number 6183 - ID: 4045 - Name: Mistscape Bracers
Scraping item number 6184 - ID: 8198 - Name: Turtle Scale Bracers
Scraping item number 6185 - ID: 15360 - Name: Trickster's Bindings
Scraping item number 6186 - ID: 4059 - Name: Glyphed Bracers
Scraping item number 6187 - ID: 6413 - Name: Chief Brigadier Bracers
Scraping item number 6188 - ID: 4259 - Name: Green Leather Bracers
Scraping item number 6189 - ID: 7437 - Name: Twilight Cuffs
Scraping item number 6190 - ID: 15577 - Name: Sparkleshell Bracers
Scraping item number 6191 - ID: 14221 - Name: Geomancer's Bracers
Scraping item number 6192 - ID: 9857 - Name: Archer's Bracers
Scraping item number 6193 - ID: 14416 - Name: Stonecloth Bindings
Scraping item number 6194 - ID: 6403 - Name: Mail Combat Armguards
Scra

In [63]:
scrapedItems = {}
itemsScraped = 0
for k, v in allItems.items():
    print("Scraping item", k, "-", v["name_enus"])
    item = GetItem(k, v["name_enus"])
    scrapedItems[k] = item
    itemsScraped += 1
    print("Items Scraped:", itemsScraped)

ing item 17588 - General's Dreadweave Gloves
Items Scraped: 688
Scraping item 29076 - Collar of the Aldor
Items Scraped: 689
Scraping item 31460 - Sha'tari Vindicator's Waistguard
Items Scraped: 690
Scraping item 27886 - Idol of the Emerald Queen
Items Scraped: 691
Scraping item 28267 - Edge of the Cosmos
Items Scraped: 692
Scraping item 28746 - Fiend Slayer Boots
Items Scraped: 693
Scraping item 5770 - Robes of Arcana
Items Scraped: 694
Scraping item 6641 - Haunting Blade
Items Scraped: 695
Scraping item 28987 - Marshal's Leather Boots
Items Scraped: 696
Scraping item 28591 - Earthsoul Leggings
Items Scraped: 697
Scraping item 6723 - Medal of Courage
Items Scraped: 698
Scraping item 28518 - Iron Gauntlets of the Maiden
Items Scraped: 699
Scraping item 32387 - Idol of the Raven Goddess
Items Scraped: 700
Scraping item 24393 - Bloody Surgeon's Mitts
Items Scraped: 701
Scraping item 21180 - Earthstrike
Items Scraped: 702
Scraping item 6975 - Whirlwind Axe
Items Scraped: 703
Scraping item

In [40]:
#URL = 'https://classic.wowhead.com/item='
#QuestCaregoryUrl = 'https://classic.wowhead.com/quests=0.'
#ZoneUrl = 'https://classic.wowhead.com/zone='
#SCRIPT_START = 'WH.Gatherer.addData('
#DATA_START = '$.extend('
#
#
#
#def GetItem(itemID):
#    if (itemID == 0):
#        return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()
#
#    print("Scraping Item", itemID)
#    url = URL+str(itemID)
#    page = requests.get(url)
#    soup = BeautifulSoup(page.content, 'html.parser')
#
#    main = soup.find(id='main-contents')
#    scripts = main.findAll('script')
#    data_container = None
#    o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, "")
#
#    #Find script tag
#    for child in scripts:
#        content = ''.join(child.contents).replace('\n    ', '').strip()
#        if content.startswith(SCRIPT_START):
#            data_container = content
#            break
#    r0 = get_region(data_container, DATA_START)
#    r1 = get_region(r0, ', ', ');')
#    data = json.loads(r1)
#    if ('sourcemore' in data):
#        usefulData = data['sourcemore'][0]
#        if ('t' in usefulData):
#            if usefulData['t'] == 1: # Kill
#                ZoneName = "Unknown Location"
#                if ('z' in usefulData):
#                    zUrl = ZoneUrl + str(usefulData['z'])
#                    pageZone = requests.get(zUrl)
#                    soupZone = BeautifulSoup(pageZone.content, 'html.parser')
#                    ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
#                    ZoneName = ZoneNameH1.text
#                dropDiv = soup.find('div', class_="whtt-dropchance")
#                dropChance = 0
#                if (dropDiv is not None):
#                    dropChance = dropDiv.text.replace("Drop Chance: ", "").replace("%", "")
#                o = Item(itemID, usefulData['ti'], usefulData['n'], True, False, 0, False, 0, dropChance, ZoneName)
#            elif usefulData['t'] == 5: # Quest
#                cUrl = QuestCaregoryUrl + str(usefulData['c'])
#                pageCat = requests.get(cUrl)
#                soupCat = BeautifulSoup(pageCat.content, 'html.parser')
#                CatNameH1 = soupCat.find('h1', class_='heading-size-1')
#                ZoneName = CatNameH1.text.replace(" Quests", "")
#                o = Item(itemID, 0, "", False, True, usefulData['ti'], False, 0, 0, ZoneName)
#            elif usefulData['t'] == 6: # Recipe
#                o = Item(itemID, 0, "", False, False, 0, True, usefulData['ti'], 0, "")
#        else:
#            if ('z' in usefulData):
#                zUrl = ZoneUrl + str(usefulData['z'])
#                pageZone = requests.get(zUrl)
#                soupZone = BeautifulSoup(pageZone.content, 'html.parser')
#                ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
#                ZoneName = ZoneNameH1.text
#                print("---------No type")
#                o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, ZoneName)
#    else:
#        print("---------No Sourcemore")
#    return o.toJson()


In [50]:
print(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))

47, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Legs": {"ID": 34381, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Feet": {"ID": 34568, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Finger": {"ID": 34213, "Obtain": {"NpcID": 25741, "NpcName": "M'uru", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "31.5", "Zone": "Sunwell Plateau"}}, "RFinger": {"ID": 29296, "Obtain": {"NpcID": 0, "NpcName": "Unknown acquisition method", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": ""}}, "Trinket": {"ID": 34473, "Obtain": {"NpcID": 24560, "NpcName": "Priestess Delrissa", "Kill": true, "Quest": false, "Que

In [21]:
newFile = open("BiSDataTBCP1-4", "w")
newFile.write(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))
newFile.close()

In [8]:
newFile = open("BiSDataTBCP1-4New", "w")
newFile.write(json.dumps(nd))
newFile.close()

In [37]:
newFile = open("AllTBCEquippableItems", "w")
newFile.write(json.dumps(scraped_items).replace('"{', '{').replace('}"', '}').replace('\\', ''))
newFile.close()